In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/MyDrive/hw3/

/content/drive/MyDrive/hw3


In [1]:
!pip install pickle5
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import pickle5 as pickle
import torch
import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertModel

/ext3/conda/hw3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('train_pp.pickle', 'rb') as f:
  train_pp = pickle.load(f)

with open('val_pp.pickle', 'rb') as f:
  val_pp = pickle.load(f)

with open('test_pp.pickle', 'rb') as f:
  test_pp = pickle.load(f)

with open('vocab.pickle', 'rb') as f:
  vocab = pickle.load(f)

In [4]:
def map_targets(vocab): 
  label_idx = {}

  idx = 0 
  for label in ['pad', 'unk']: 
    label_idx[label] = idx
    idx += 1

  for label in vocab: 
    if label not in ['pad', 'bos', 'eos', 'unk']: 
      label_idx[label] = idx
      idx += 1
  
  return label_idx 

In [5]:
rel_pos_idxs = map_targets(vocab['rel_pos_set'])
dep_label_idxs = map_targets(vocab['dep_label_set'])

In [6]:
with open('rel_pos_idxs.pickle', 'wb') as handle:
    pickle.dump(rel_pos_idxs, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('dep_label_idxs.pickle', 'wb') as handle:
    pickle.dump(dep_label_idxs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
dep_label_idxs

{'pad': 0,
 'unk': 1,
 'nmod': 2,
 'expl': 3,
 'appos': 4,
 'xcomp': 5,
 'conj': 6,
 'det': 7,
 'vocative': 8,
 'obj': 9,
 'goeswith': 10,
 'advcl': 11,
 'mark': 12,
 'aux': 13,
 'nmod:tmod': 14,
 'cc:preconj': 15,
 'nsubj': 16,
 'case': 17,
 'acl': 18,
 'iobj': 19,
 'csubj:pass': 20,
 'det:predet': 21,
 'compound:prt': 22,
 'parataxis': 23,
 'dislocated': 24,
 'acl:relcl': 25,
 'obl': 26,
 'reparandum': 27,
 'csubj': 28,
 'cc': 29,
 'aux:pass': 30,
 'discourse': 31,
 'root': 32,
 'nummod': 33,
 'list': 34,
 'advmod': 35,
 'flat': 36,
 'ccomp': 37,
 'nsubj:pass': 38,
 'nmod:npmod': 39,
 'orphan': 40,
 'obl:tmod': 41,
 'obl:npmod': 42,
 'nmod:poss': 43,
 'cop': 44,
 'punct': 45,
 'compound': 46,
 'fixed': 47,
 'dep': 48,
 'amod': 49}

In [8]:
class distilBERT_FT(nn.Module):

    def __init__(self, pre_model, hidden_dim, n_rel_classes, n_dep_classes):
      super().__init__()
      self.pre_model = pre_model
      self.rel_project = nn.Linear(hidden_dim, n_rel_classes)
      self.dep_project = nn.Linear(hidden_dim, n_dep_classes)

    def forward(self, input):
      #print(f"input size: {input['input_ids'].size()}")

      # hidden_states = self.pre_model(input['input_ids']).last_hidden_state # [batch_size, seq_len, hidden_dim]
      hidden_states = self.pre_model(**input).last_hidden_state
      #print(f'hidden_states : {hidden_states.size()}')

      rel_out = self.rel_project(hidden_states)
      #print(f'rel_out size: {rel_out.size()}')
      #print(f'rel_out: {rel_out}')

      dep_out = self.dep_project(hidden_states)
      #print(f'dep_out size: {dep_out.size()}')
      #print(f'dep_out: {dep_out}')

      return rel_out, dep_out

In [9]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [12]:
from torch.utils.data import DataLoader

def pad_list_of_tensors(list_of_tensors, pad_token):
  # print('list of tensors: ', list_of_tensors)
  max_length = max([t.size(-1) for t in list_of_tensors])
  padded_list = []

  # print('max length:', max_length)
  
  for t in list_of_tensors:
    # print('t:', t.size())
    padded_tensor = torch.cat((t, torch.tensor([pad_token]*(max_length - t.size(-1)), dtype=torch.long)), dim = -1)
    # print('padded_tensor: ', padded_tensor)
    # print('padded_tensor size: ', padded_tensor.size())
    padded_list.append(padded_tensor)
      
  padded_tensor = torch.stack(padded_list)
  #print('padded_tensor size: ', padded_tensor.size())
  return padded_tensor

def pad_collate_fn(batch):
  # print('prepadded batch: ', batch)
  input_sent_list = [s['text'] for s in batch] # not adding bos and eos because tokenizer does that for us 
  #print(input_list)
  input_token_list = [s['tokens'] for s in batch] # not adding bos and eos because I dont end up using this 
  #print(input_list)
  rel_list = [s['rel_pos'] for s in batch]
  #print(rel_list)
  dep_list = [s['dep_label'] for s in batch]
  #print(dep_list)

  rel_mapped_list = []
  dep_mapped_list = []

  for rels in rel_list: 
    temp_rel_list = []

    for rel in rels: 
      temp_rel_list.append(rel_pos_idxs[rel])

    rel_mapped_list.append(torch.tensor(temp_rel_list))

  for deps in dep_list: 
    temp_dep_list = []

    for dep in deps: 
      temp_dep_list.append(dep_label_idxs[dep])

    dep_mapped_list.append(torch.tensor(temp_dep_list))
    
  # print(rel_mapped_list)
  # print(dep_mapped_list)

  rel_pad_idx = rel_pos_idxs.get('pad')
  dep_pad_idx = dep_label_idxs.get('pad')

  # print(rel_pad_idx)
  # print(dep_pad_idx)
    
  # padding the input is probably not necessary here as we will be using bert tokenizer to pad the input sentences later on 
  # input_tensor = pad_list_of_tensors(input_list, pad_token)

  rel_tensor = pad_list_of_tensors(rel_mapped_list, rel_pad_idx)
  dep_tensor = pad_list_of_tensors(dep_mapped_list, dep_pad_idx)
    
  return input_sent_list, input_token_list, rel_tensor, dep_tensor

In [13]:
BATCH_SIZE = 32
dataloader_dict = {}

# change shuffle back to True later
dataloader_dict['train'] = DataLoader(train_pp, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_collate_fn)
dataloader_dict['val'] = DataLoader(val_pp, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_collate_fn)
dataloader_dict['test'] = DataLoader(test_pp, batch_size=BATCH_SIZE, shuffle=True, collate_fn=pad_collate_fn)

In [58]:
def filter_scores_and_preds(input_sents, input_tokens, output, target_size, n_classes):
  filtered_scores = torch.zeros((output.size()[0], target_size, n_classes))
  #print('filtered_scores size of zeros: ', filtered_scores.size())

  preds_list = []
  for idx, sentence in enumerate(output): 
    bert_tokens = tokenizer.tokenize(input_sents[idx])
    bert_tokens = ['cls'] + bert_tokens + ['sep']
    # print('bert_tokens: ', bert_tokens)
    # print('bert_tokens size: ', len(bert_tokens))
    # print('UD tokens: ', input_tokens[idx])
    # print('UD tokens: ', len(input_tokens[idx]))
    # print('max seq len: ', target_size)

    hyphen_prev = False 
    preds = []
    filtered_idx = []

    for t_idx, token in enumerate(bert_tokens):
      if t_idx == 0 or t_idx == len(bert_tokens) - 1: 
        continue
      elif token.startswith('##'): 
        continue 
      elif token == '-': 
        hyphen_prev = True 
        continue
      elif hyphen_prev == True: 
        hyphen_prev = False 
        continue 
      else: 
        filtered_idx.append(t_idx)

    if len(filtered_idx) > len(input_tokens[idx]): 
      filtered_idx = filtered_idx[:len(input_tokens[idx])]

    #print('filtered_idx: ', filtered_idx)
    #print('num of idx in filtered_idx: ', len(filtered_idx))

    fs_idx = 0 
    for good_idx in filtered_idx:
       filtered_scores[idx][fs_idx] = sentence[good_idx]
       preds.append(torch.argmax(sentence[good_idx]).item())
       fs_idx += 1
      
    #print('filtered_scores: ', filtered_scores[idx])
    
    preds = preds + ([0] * (target_size - len(preds)))

    preds_list.append(preds)

  preds_list = torch.tensor(preds_list)
  # #print('preds_list: ', preds_list)
    
  #print('filtered_scores size: ', filtered_scores.size())
  return filtered_scores, preds_list

In [20]:
from tqdm import tqdm

In [21]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [70]:
n_rel_classes = len(rel_pos_idxs)
n_dep_classes = len(dep_label_idxs)

classifier = distilBERT_FT(model, 768, n_rel_classes, n_dep_classes).to(device)

criterion = torch.nn.CrossEntropyLoss(ignore_index = rel_pos_idxs['pad']).to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

def train(model, num_epochs, criterion, optimizer, lam, device): 
  pbar = tqdm(range(num_epochs))
  val_accs = []

  for epoch in pbar:
    print('EPOCH: ', epoch)
    total_train_loss = 0.0
    train_rel_accs = []
    train_dep_accs = []

    total_val_loss = 0.0
    val_rel_accs = []
    val_dep_accs = []

    model.train()
    for input_sents, input_tokens, rel_target, dep_target in tqdm(dataloader_dict['train']): 
      # encoded_input['input_ids'], encoded_input['attention_mask']
      encoded_input = tokenizer(input_sents, return_tensors='pt', padding=True).to(device)
      rel_output, dep_output = model(encoded_input)

      #print('rel_target size: ', rel_target.size())

      filtered_rel_scores, filtered_rel_preds = filter_scores_and_preds(input_sents, input_tokens, rel_output, rel_target.size()[1], n_rel_classes)
      filtered_dep_scores, filtered_dep_preds = filter_scores_and_preds(input_sents, input_tokens, dep_output, dep_target.size()[1], n_dep_classes)

      rel_loss = criterion(filtered_rel_scores.permute(0,2,1).to(device), rel_target.to(device))
      dep_loss = criterion(filtered_dep_scores.permute(0,2,1).to(device), dep_target.to(device))
    
      loss = (lam * rel_loss) + ((1 - lam) * dep_loss)
      #print('loss: ', loss)
      total_train_loss += loss

      # rel_acc = torch.eq(filtered_rel_preds, rel_target).sum() / (rel_target.size()[0] * rel_target.size()[1])
      rel_no_pad_cnt = 0 
      rel_correct = 0
      for s_idx, sent in enumerate(filtered_rel_preds): 
        for t_idx, token in enumerate(sent):
          if token != 0: 
            rel_no_pad_cnt += 1
            if token == rel_target[s_idx][t_idx]:
              rel_correct += 1
          
      train_rel_accs.append(rel_correct / rel_no_pad_cnt)

      # dep_acc = torch.eq(filtered_dep_preds, dep_target).sum() / (dep_target.size()[0] * dep_target.size()[1])
      dep_no_pad_cnt = 0 
      dep_correct = 0
      for s_idx, sent in enumerate(filtered_dep_preds): 
        for t_idx, token in enumerate(sent):
          if token != 0: 
            dep_no_pad_cnt += 1
            if token == dep_target[s_idx][t_idx]:
              dep_correct += 1
    
      train_dep_accs.append(dep_correct / dep_no_pad_cnt)

      optimizer.zero_grad()
      loss.backward()
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type=2)
      optimizer.step() 

    model.eval()
    for input_sents, input_tokens, rel_target, dep_target in tqdm(dataloader_dict['val']): 
      encoded_input = tokenizer(input_sents, return_tensors='pt', padding=True).to(device)
      rel_output, dep_output = model(encoded_input)

      filtered_rel_scores, filtered_rel_preds = filter_scores_and_preds(input_sents, input_tokens, rel_output, rel_target.size()[1], n_rel_classes)
      filtered_dep_scores, filtered_dep_preds = filter_scores_and_preds(input_sents, input_tokens, dep_output, dep_target.size()[1], n_dep_classes)

      rel_loss = criterion(filtered_rel_scores.permute(0,2,1).to(device), rel_target.to(device))
      dep_loss = criterion(filtered_dep_scores.permute(0,2,1).to(device), dep_target.to(device))

      val_loss = (lam * rel_loss) + ((1 - lam) * dep_loss)
      #print('val loss: ', val_loss)
      total_val_loss += val_loss

      # rel_acc = torch.eq(filtered_rel_preds, rel_target).sum() / (rel_target.size()[0] * rel_target.size()[1])
      rel_no_pad_cnt = 0 
      rel_correct = 0
      for s_idx, sent in enumerate(filtered_rel_preds): 
        for t_idx, token in enumerate(sent):
          if token != 0: 
            rel_no_pad_cnt += 1
            if token == rel_target[s_idx][t_idx]:
              rel_correct += 1
          
      val_rel_accs.append(rel_correct / rel_no_pad_cnt)

      # dep_acc = torch.eq(filtered_dep_preds, dep_target).sum() / (dep_target.size()[0] * dep_target.size()[1])
      dep_no_pad_cnt = 0 
      dep_correct = 0
      for s_idx, sent in enumerate(filtered_dep_preds): 
        for t_idx, token in enumerate(sent):
          if token != 0: 
            dep_no_pad_cnt += 1
            if token == dep_target[s_idx][t_idx]:
              dep_correct += 1
    
      val_dep_accs.append(dep_correct / dep_no_pad_cnt)

    print('avg train loss: ', total_train_loss / len(dataloader_dict['train']))
    print('avg train rel_pos acc: ', sum(train_rel_accs) / len(train_rel_accs))
    print('avg train dep_label acc: ', sum(train_dep_accs) / len(train_dep_accs))

    print('avg val loss: ', total_val_loss / len(dataloader_dict['val']))
    print('avg val rel_pos acc: ', sum(val_rel_accs) / len(val_rel_accs))
    print('avg val dep_label acc: ', sum(val_dep_accs) / len(val_dep_accs))
    if epoch == num_epochs-1: 
      val_accs.append({'rel_pos': sum(val_rel_accs) / len(val_rel_accs),  'dep_label': sum(val_dep_accs) / len(val_dep_accs)})

  print('SAVING MODEL')
  torch.save(model.state_dict(), f'bert-parser-{lam}.pt')
  return val_accs

In [65]:
train(classifier, 1, criterion, optimizer, 0.25, device)

  0%|          | 0/1 [00:00<?, ?it/s]

EPOCH:  0


rel_target size:  torch.Size([32, 65])
loss:  tensor(4.1995, device='cuda:0', grad_fn=<AddBackward0>)


rel_target size:  torch.Size([32, 51])
loss:  tensor(3.7788, device='cuda:0', grad_fn=<AddBackward0>)


rel_target size:  torch.Size([32, 54])
loss:  tensor(3.3829, device='cuda:0', grad_fn=<AddBackward0>)


...

val loss:  tensor(1.0260, device='cuda:0', grad_fn=<AddBackward0>)
val loss:  tensor(0.9695, device='cuda:0', grad_fn=<AddBackward0>)


val loss:  tensor(0.8739, device='cuda:0', grad_fn=<AddBackward0>)
val loss:  tensor(1.1136, device='cuda:0', grad_fn=<AddBackward0>)


val loss:  tensor(0.5114, device='cuda:0', grad_fn=<AddBackward0>)
val loss:  tensor(0.7134, device='cuda:0', grad_fn=<AddBackward0>)


100%|██████████| 1/1 [00:46<00:00, 46.10s/it]


val loss:  tensor(0.5667, device='cuda:0', grad_fn=<AddBackward0>)
avg train loss:  tensor(1.0447, device='cuda:0', grad_fn=<DivBackward0>)
avg train rel_pos acc:  0.5522308882106988
avg train dep_label acc:  0.8385585356986963
avg val loss:  tensor(0.8272, device='cuda:0', grad_fn=<DivBackward0>)
avg val rel_pos acc:  0.630969700442471
avg val dep_label acc:  0.8473900227382117


In [35]:
train(classifier, 1, criterion, optimizer, 0.25, device)

  0%|          | 0/1 [00:00<?, ?it/s]

EPOCH:  0


loss:  tensor(4.1841, device='cuda:0', grad_fn=<AddBackward0>)


loss:  tensor(4.0369, device='cuda:0', grad_fn=<AddBackward0>)
loss:  tensor(3.8734, device='cuda:0', grad_fn=<AddBackward0>)


loss:  tensor(3.6675, device='cuda:0', grad_fn=<AddBackward0>)
loss:  tensor(3.4427, device='cuda:0', grad_fn=<AddBackward0>)


...

val loss:  tensor(1.0818, device='cuda:0', grad_fn=<AddBackward0>)
val loss:  tensor(0.8726, device='cuda:0', grad_fn=<AddBackward0>)


val loss:  tensor(0.8514, device='cuda:0', grad_fn=<AddBackward0>)
val loss:  tensor(0.9828, device='cuda:0', grad_fn=<AddBackward0>)
val loss:  tensor(0.5621, device='cuda:0', grad_fn=<AddBackward0>)


100%|██████████| 1/1 [00:37<00:00, 37.36s/it]


avg train loss:  tensor(1.6790, device='cuda:0', grad_fn=<DivBackward0>)
avg train rel_pos acc:  0.42843728613060955
avg train dep_label acc:  0.6419149971273127
avg val loss:  tensor(1.0104, device='cuda:0', grad_fn=<DivBackward0>)
avg val rel_pos acc:  0.5549146758176737
avg val dep_label acc:  0.7988441375770617
SAVING MODEL


[{'rel_pos': 0.5549146758176737, 'dep_label': 0.7988441375770617}]

In [39]:
lam_val_accs = []
for lam in [0.25, 0.5, 0.75]:
    model = DistilBertModel.from_pretrained("distilbert-base-uncased")
    classifier = distilBERT_FT(model, 768, n_rel_classes, n_dep_classes).to(device)

    criterion = torch.nn.CrossEntropyLoss(ignore_index = rel_pos_idxs['pad']).to(device)
    optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)
    val_accs = train(classifier, 3, criterion, optimizer, lam, device)
    lam_val_accs.append(val_accs)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/3 [00:00<?, ?it/s]

EPOCH:  0


 33%|███▎      | 1/3 [00:33<01:07, 33.68s/it]

avg train loss:  tensor(1.6780, device='cuda:0', grad_fn=<DivBackward0>)
avg train rel_pos acc:  0.4241927727345834
avg train dep_label acc:  0.6421994190668313
avg val loss:  tensor(1.0459, device='cuda:0', grad_fn=<DivBackward0>)
avg val rel_pos acc:  0.5439136290436235
avg val dep_label acc:  0.7866831156069918
EPOCH:  1


 67%|██████▋   | 2/3 [01:06<00:33, 33.38s/it]

...

avg train loss:  tensor(2.0098, device='cuda:0', grad_fn=<DivBackward0>)
avg train rel_pos acc:  0.49480344136593823
avg train dep_label acc:  0.4785377302243596
avg val loss:  tensor(1.3007, device='cuda:0', grad_fn=<DivBackward0>)
avg val rel_pos acc:  0.6490743061059027
avg val dep_label acc:  0.6974944783414159
EPOCH:  1


 67%|██████▋   | 2/3 [01:06<00:33, 33.29s/it]

avg train loss:  tensor(1.1115, device='cuda:0', grad_fn=<DivBackward0>)
avg train rel_pos acc:  0.6810296754622306
avg train dep_label acc:  0.7423418436356254
avg val loss:  tensor(1.0259, device='cuda:0', grad_fn=<DivBackward0>)
avg val rel_pos acc:  0.7056174590790745
avg val dep_label acc:  0.769180724575546
EPOCH:  2


100%|██████████| 3/3 [01:39<00:00, 33.23s/it]


avg train loss:  tensor(0.8406, device='cuda:0', grad_fn=<DivBackward0>)
avg train rel_pos acc:  0.742553238936682
avg train dep_label acc:  0.8040866460139139
avg val loss:  tensor(0.9435, device='cuda:0', grad_fn=<DivBackward0>)
avg val rel_pos acc:  0.7199073611090989
avg val dep_label acc:  0.8009836938271249
SAVING MODEL


In [40]:
print(lam_val_accs)

[[{'rel_pos': 0.66389582946543, 'dep_label': 0.8345714217217187}], [{'rel_pos': 0.706399152137012, 'dep_label': 0.83666102417964}], [{'rel_pos': 0.7199073611090989, 'dep_label': 0.8009836938271249}]]


In [71]:
lam_val_accs = []
for lam in [0.25, 0.5, 0.75]:
    model = DistilBertModel.from_pretrained("distilbert-base-uncased")
    classifier = distilBERT_FT(model, 768, n_rel_classes, n_dep_classes).to(device)

    criterion = torch.nn.CrossEntropyLoss(ignore_index = rel_pos_idxs['pad']).to(device)
    optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)
    val_accs = train(classifier, 3, criterion, optimizer, lam, device)
    lam_val_accs.append(val_accs)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/3 [00:00<?, ?it/s]

EPOCH:  0


 33%|███▎      | 1/3 [00:45<01:30, 45.03s/it]

avg train loss:  tensor(1.6797, device='cuda:0', grad_fn=<DivBackward0>)
avg train rel_pos acc:  0.42966262638276725
avg train dep_label acc:  0.6535627742033372
avg val loss:  tensor(1.0556, device='cuda:0', grad_fn=<DivBackward0>)
avg val rel_pos acc:  0.5461990177780157
avg val dep_label acc:  0.7985557817847743
EPOCH:  1


 67%|██████▋   | 2/3 [01:29<00:44, 44.57s/it]

...

avg train loss:  tensor(1.9675, device='cuda:0', grad_fn=<DivBackward0>)
avg train rel_pos acc:  0.517218328322159
avg train dep_label acc:  0.4975170520184258
avg val loss:  tensor(1.2829, device='cuda:0', grad_fn=<DivBackward0>)
avg val rel_pos acc:  0.6620825959559354
avg val dep_label acc:  0.7148070779451117
EPOCH:  1


 67%|██████▋   | 2/3 [01:29<00:45, 45.04s/it]

avg train loss:  tensor(1.0714, device='cuda:0', grad_fn=<DivBackward0>)
avg train rel_pos acc:  0.7044930597015716
avg train dep_label acc:  0.7607579339060323
avg val loss:  tensor(1.0081, device='cuda:0', grad_fn=<DivBackward0>)
avg val rel_pos acc:  0.715977074534533
avg val dep_label acc:  0.7843808159701016
EPOCH:  2


100%|██████████| 3/3 [02:14<00:00, 44.98s/it]


avg train loss:  tensor(0.8239, device='cuda:0', grad_fn=<DivBackward0>)
avg train rel_pos acc:  0.7603411095272384
avg train dep_label acc:  0.8216181225699373
avg val loss:  tensor(0.9348, device='cuda:0', grad_fn=<DivBackward0>)
avg val rel_pos acc:  0.7350539435318791
avg val dep_label acc:  0.8128642721196735
SAVING MODEL


In [72]:
lam_val_accs 

[[{'rel_pos': 0.6714941035397294, 'dep_label': 0.852764262844792}],
 [{'rel_pos': 0.7238580304412094, 'dep_label': 0.8473114098190778}],
 [{'rel_pos': 0.7350539435318791, 'dep_label': 0.8128642721196735}]]